<a href="https://colab.research.google.com/github/dunomi/zw/blob/master/z9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import statsmodels.api as sm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')
USA_state_names= pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/states.csv')
USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)
USA

In [0]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
  
USAdata=pd.merge(USAcases,USAweather)

In [0]:
USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

#USAcumdata.loc[USAcumdata.Przyrosty > 50, 'Przyrosty'] = max(USAcumdata.loc[USAcumdata.Przyrosty < 50].Przyrosty)

In [0]:
for i in range(len(USAdata)):
  USAdata.loc[i, 'Temperatura_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Temperatura[i][0:6].astype(float))[0,1]
  USAdata.loc[i, 'Wilgotnosc_cor']=np.corrcoef(USAdata.Przyrosty[i],USAdata.Wilgotnosc[i][0:6].astype(float))[0,1]

  USAdata.loc[i, 'Temperatura_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Temperatura'][i][0:6].tolist())).fit().params[1]
  USAdata.loc[i, 'Wilgotnosc_cor_wsp_ols']=sm.OLS(USAdata['Przyrosty'][i].tolist(),sm.add_constant(USAdata['Wilgotnosc'][i][0:6].tolist())).fit().params[1]

  USAdata.loc[i, 'Przyrosty_wykres_osY']=(max(USAdata.Przyrosty[i])-min(USAdata.Przyrosty[i]))/2
  USAdata.loc[i, 'Temperatura_wykres_osX']=(max(USAdata.Temperatura[i])-min(USAdata.Temperatura[i]))/2
  USAdata.loc[i, 'Wilgotnosc_wykres_osX']=(max(USAdata.Wilgotnosc[i])-min(USAdata.Wilgotnosc[i]))/2

In [0]:
USAdata.to_csv(index=True)
writer = pd.ExcelWriter('hey.xlsx')
USAdata.to_excel(writer)
writer.save()

In [0]:
from google.colab import drive
drive.mount('/content/drive')